unzip json files in to a folder in google drive cloud

---



In [2]:
# from zipfile import ZipFile
# with ZipFile('drive/MyDrive/datasets.zip', 'r') as zipObj:
  # zipObj.extractall('drive/MyDrive/test')

In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)

In [4]:
path = '/content/drive/MyDrive/test/Dataset Yelp/'

In [5]:
df_business = pd.read_json(f'{path}business.json', lines=True)

In [6]:
business_index = pd.DataFrame()
business_index['id'] = list(range(1, len(df_business) + 1, 1))
business_index['business_id'] = df_business['business_id'].copy()
business_index.head()

,id,business_id
0,1,Pns2l4eNsfO8kk83dixA6A
1,2,mpf3x-BjTdTEA3yCZrAYPw
2,3,tUFrWirKiKi_TAnsVWINQQ
3,4,MTSW4McQd7CbVtyjqoe9mw
4,5,mWMc6_wTdE0EUBKIGXDVfA


In [7]:
df_business.drop(['business_id'], axis=1, inplace=True)
new_col = list(range(1, len(df_business) +1, 1))
df_business.insert(loc = 0, column = 'business_id', value = new_col) 

In [8]:
df_business['city'] = df_business['city'].apply(lambda x: x.lower())
df_business['city'] = df_business['city'].str.replace(',','')
df_business['city'] = df_business['city'].str.replace('.','')
df_business['city'] = df_business['city'].apply(lambda x: x.strip())
df_business['city'] = df_business['city'].apply(lambda x: " ".join(x.split()))
df_business['city'] = df_business['city'].apply(lambda x: x.title())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
df_business['categories'] = df_business['categories'].fillna('0')

In [10]:
cat_list = ['Restaurants', 'Home Services', 'Auto Services', 'Auto Repair', 'Beauty & Spas', 'Food', 'Nightlife', 'Hotels', 'Local Services', 'Pets']
def validator(value):
  if value in cat_list:
    return 1000
  else:
    return 0
print(validator('Restaurants'))

def cat_to_list(value):
  value = value.split(', ')
  value.sort(key=validator, reverse=True)
  return ', '.join(value)

  
df_business['categories'] = df_business['categories'].apply(lambda x: cat_to_list(x))

1000


In [11]:
# df[df['A'].str.contains("hello")]
# df_business['categories'][df_business['categories'].str.contains('Restaurant')]
# df.sort_values(by=['col1'])
city_dict = {'Belle Chase': 'Belle Chasse',
             'Abington Township': 'Abington',
             'Ashland City': 'Ashland',
             'Bellefontaine Neighbors': 'Bellefontaine',
             'Bellville': 'Belleville',
             'Belleair Blf': 'Belleair Bluffs',
             'Bethel Township': 'Bethel',
             'Bensalem Pa': 'Bensalem',
             'Bensalem Township': 'Bensalem',
             'Boise City': 'Boise',
             'Burlington Township': 'Burlington',
             "Carney'S Point": 'Carneys Point',
             'Cedar Brook': 'Cedarbrook',
             '/': '',
             'Conshohoeken':'Conshohocken',
             'Delran Township': 'Delran',
             'Delran Twp': 'Delran',
             'Concord Township': 'Concord',
             'Deptford Township': 'Deptford',
             'Eastampton Township': 'Eastampton',
             ' Township': '',
             'Fairview Hts': 'Fairview',
             '-': '',
             ' City': '',
             ' Twp': '',
             'Bch': 'Beach',
             'Land O Lakes':"Land O'Lakes",
             "Land O' Lakes":"Land O'Lakes",
             'Mccordsville': 'Mc Cordsville',
             'Metarie': 'Metairie',
             'Mt Laurel Twp Nj': 'Mt Laurel',
             'Sqaure': 'Square',
             'O Fallon': "O'Fallon",
             "O' Fallon": "O'Fallon",
             'Phila': 'Philadelphia',
             'Philadephia': 'Philadelphia',
             'Philly': 'Philadelphia',
             'Redingtn Shor': 'Redington Shore',
             'Redington Shores': 'Redington Shore',
             'Riverview Fl': 'Riverview',
             'Saintt': 'Saint',
             'Tierre': 'Tierra',
             "Town 'N' Country": 'Town & Country',
             'Town And Country': 'Town & Country',
             'Town N Country': 'Town & Country',
             'Tuscon': 'Tucson'
             }

df_business = df_business.replace({'city': city_dict})
df_restaurants = df_business[df_business['categories'].str.contains('Restaurant')]

In [12]:
df_city_state = df_business[['city', 'state']].drop_duplicates().copy()
new_col2 = list(range(1, len(df_city_state)+1, 1))
df_city_state.insert(loc = 0, column = 'city_state_id', value = new_col2) 
df_city_state.head(2)

,city_state_id,city,state
0,1,Santa Barbara,CA
1,2,Affton,MO


In [13]:
df2 = pd.merge(df_business, df_city_state, left_on=['city', 'state'], right_on=['city', 'state'], how='left')

In [14]:
df_business['city_state_id'] = df2['city_state_id']

In [15]:
# # redondear latitud y longitud para reducir ciudades
# import math
# math_aux = 100
# df_business['latitude']  = df_business['latitude'].apply(lambda x: math.floor(x*math_aux)/math_aux)
# df_business['longitude'] = df_business['longitude'].apply(lambda x: math.floor(x*math_aux)/math_aux)

In [16]:
def cat_to_col(value):
  aux_dict = {}
  x = value.split(', ')
  for i in x:
    aux_dict[i] = 1
  return aux_dict

In [17]:
def dict_to_columns(df_i, column):
  df_i[column] = df_i[column].fillna('{}')
  df_i.reset_index(inplace=True)
  df_o = df_i.join(pd.json_normalize(df_i.pop(column)))
  if 'level_0' in df_o.columns:
    df_o.drop(['level_0'], axis=1, inplace=True)

  return df_o

In [18]:
# funcion para tratar la columna atributos y obtener solo valores verdaderos del diccionario atributos
def attr_to_list(i):
  attributes_list = []
  try:
    for j in i:
      if i[j] == 'True' or i[j] == '1' or i[j] == "u'free'":
        attributes_list.append(j)
    return ', '.join(attributes_list)
  except:
    return ''

In [19]:
df_restaurants['attributes'] = df_restaurants['attributes'].apply(lambda x: attr_to_list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_business['attributes'] = df_business['attributes'].apply(lambda x: attr_to_list(x))

df1: business_id, name, address, postal_code, latitude, longitude => business_address

df2: city, state => business_cities

df3: attributes, categories => business_categories

df4: stars, review_count, is_open => business_review

df5: hours => busines_hours

In [21]:
df_hours = pd.DataFrame()
df_hours['hours'] = df_business['hours'].copy()
df_hours['aux'] = df_hours['hours'].astype('str')
df_hours = dict_to_columns(df_hours, 'hours')
df_hours = df_hours.fillna(0)
df_hours.drop(['index'], axis=1, inplace=True)
df_hours = df_hours.drop_duplicates()
df_hours.insert(loc = 0, column = 'hours_id', value = list(range(1, len(df_hours)+1, 1)))

In [22]:
df_business['hours'] = df_business['hours'].astype('str')

In [23]:
df_hours.columns

Index(['hours_id', 'aux', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday'],
      dtype='object')

In [24]:
df3=pd.merge(df_business,df_hours, left_on='hours', right_on='aux', how='left')
df3.drop(['hours','aux','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], axis=1, inplace=True)
df_business['hours_id'] = df3['hours_id']

In [25]:
df_business.drop(['hours'], axis=1, inplace=True)

In [26]:
df_business['attributes'] = df_business['attributes'].astype('str')
df_business.dtypes

business_id        int64
name              object
address           object
city              object
state             object
postal_code       object
latitude         float64
longitude        float64
stars            float64
review_count       int64
is_open            int64
attributes        object
categories        object
city_state_id      int64
hours_id           int64
dtype: object

In [27]:
df_attributes = pd.DataFrame()
df_attributes['attributes'] = df_business['attributes']
df_attributes = df_attributes.drop_duplicates()
df_attributes.insert(loc = 0, column = 'attributes_id', value = list(range(1, len(df_attributes)+1, 1)))
df_attributes.head()

,attributes_id,attributes
0,1,ByAppointmentOnly
1,2,BusinessAcceptsCreditCards
2,3,"BikeParking, BusinessAcceptsCreditCards, Wheel..."
3,4,"BikeParking, RestaurantsPriceRange2, Restauran..."
4,5,"BusinessAcceptsCreditCards, WheelchairAccessib..."


In [28]:
df5 = pd.merge(df_business, df_attributes, left_on=['attributes'], right_on=['attributes'], how='left')
df_business['attributes_id'] = df5['attributes_id']

In [29]:
df_categories = pd.DataFrame()
df_categories['categories'] = df_business['categories']
df_categories = df_categories.drop_duplicates()
df_categories.insert(loc = 0, column = 'categories_id', value = list(range(1, len(df_categories)+1, 1)))
df_categories.head(3)

,categories_id,categories
0,1,"Doctors, Traditional Chinese Medicine, Naturop..."
1,2,"Local Services, Shipping Centers, Notaries, Ma..."
2,3,"Department Stores, Shopping, Fashion, Home & G..."


In [30]:
df6 = pd.merge(df_business, df_categories, left_on='categories', right_on='categories', how='left')
df_business['categories_id'] = df6['categories_id']

In [31]:
df_business.drop(['city', 'state', 'attributes','categories'], axis=1, inplace=True)

In [32]:
df_business.head(3)

,business_id,name,address,postal_code,latitude,longitude,stars,review_count,is_open,city_state_id,hours_id,attributes_id,categories_id
0,1,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",93101,34.426679,-119.711197,5.0,7,0,1,1,1,1
1,2,The UPS Store,87 Grasso Plaza Shopping Center,63123,38.551126,-90.335695,3.0,15,1,2,2,2,2
2,3,Target,5255 E Broadway Blvd,85711,32.223236,-110.880452,3.5,22,0,3,3,3,3


In [33]:
df_city_state.head(3)

,city_state_id,city,state
0,1,Santa Barbara,CA
1,2,Affton,MO
2,3,Tucson,AZ


In [34]:
df_hours.head(3)

,hours_id,aux,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,None,0,0,0,0,0,0,0
1,2,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,0
2,3,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0


In [35]:
df_attributes.head(3)

,attributes_id,attributes
0,1,ByAppointmentOnly
1,2,BusinessAcceptsCreditCards
2,3,"BikeParking, BusinessAcceptsCreditCards, Wheel..."


In [36]:
df_categories.head(3)

,categories_id,categories
0,1,"Doctors, Traditional Chinese Medicine, Naturop..."
1,2,"Local Services, Shipping Centers, Notaries, Ma..."
2,3,"Department Stores, Shopping, Fashion, Home & G..."


In [37]:
df_hours.drop(['aux'], axis=1, inplace=True)

In [38]:
hours_aux = df_hours.columns.tolist()[1:]
for i in hours_aux:
  df_hours[i] = df_hours[i].astype('str')

In [39]:
# df_business.to_parquet('/content/drive/MyDrive/output/business.parquet.gzip', compression='gzip')
output_path = '/content/drive/MyDrive/output/'
df_business.to_parquet(f'{output_path}business.parquet.gzip', compression='gzip')
df_hours.to_parquet(f'{output_path}business_hours.parquet.gzip', compression='gzip')
df_attributes.to_parquet(f'{output_path}business_attributes.parquet.gzip', compression='gzip')
df_categories.to_parquet(f'{output_path}business_categories.parquet.gzip', compression='gzip')
df_categories.to_parquet(f'{output_path}business_categories.parquet.gzip', compression='gzip')
df_city_state.to_parquet(f'{output_path}business_city_state.parquet.gzip', compression='gzip')
business_index.to_csv('/content/drive/MyDrive/output/business_index.csv', index=False)

analisis de rubros

In [40]:
lista_rubros = []
for i in df_business['categories']:
  for j in i.split(', '):
    if j not in lista_rubros:
      lista_rubros.append(j)

KeyError: ignored

categorias: 
restaurantes
bares
hoteles


In [ ]:
len(lista_rubros)

In [ ]:
import collections

d = {2:3, 1:89, 4:5, 3:0}
od = collections.OrderedDict(sorted(d.items()))
type(od)

for k, v in od.iteritems():
  print(k, v)

In [ ]:
import datetime

now = datetime.datetime.now()
print(now.strftime("%A"))
print(now)